In [72]:
import re
import nltk
from nltk.tokenize import word_tokenize
import pymorphy2
from tqdm.auto import tqdm
morph = pymorphy2.MorphAnalyzer()

In [137]:
from pymystem3 import Mystem
import string
m = Mystem()

In [143]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,

    Doc
)
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
segmenter = Segmenter()

## Создаем список с текстами и список с заголовками

In [99]:
poems = []
poems_titles = []
for i in range(0, 96):
    with open('autoproj/poem' + str(i+1) + '.txt', 'r', encoding='utf-8') as pm:
        poem = pm.read().split('\n')
        poems_titles.append(poem[0])
        poems.append(poem[1].rstrip('  '))
        

## Создаем словарь вида {text_index: {sentence_index: [sentense, lemmatized_sentence]}} и записываем в датабазу

In [121]:
sentences_in_text = {}
for j in tqdm(range(len(poems))): 
    poem_sntncs = poems[j].split('  ') 
    poem_sentences = {}
    for i in range(len(poem_sntncs)):         
        tokenized_sentence = word_tokenize(poem_sntncs[i], 'russian') 
        sentence = '' 
        for word in tokenized_sentence: 
            w = morph.parse(word)[0].normal_form 
            sentence += w + ' ' 
        poem_sentences[i+1] = [poem_sntncs[i], sentence]
    sentences_in_text[j+1] = poem_sentences

In [123]:
sentences_in_text[1]

{1: ['В небольшом квартале к западу от Вашингтон-сквера улицы перепутались и переломались в короткие полоски, именуемые проездами.',
  'в небольшой квартал к запасть от вашингтон-сквер улица перепутаться и переломаться в короткий полоска , именовать проезд . '],
 2: ['Эти проезды образуют странные углы и кривые линии.',
  'этот проезд образовать странный угол и кривая линия . '],
 3: ['Одна улица там даже пересекает самое себя раза два.',
  'один улица там даже пересекать самый себя раз два . '],
 4: ['Некоему художнику удалось открыть весьма ценное свойство этой улицы.',
  'некий художник удаться открыть весьма ценный свойство этот улица . '],
 5: ['Предположим, сборщик из магазина со счетом за краски, бумагу и холст повстречает там самого себя, идущего восвояси, не получив ни единого цента по счету!',
  'предположим , сборщик из магазин с счёт за краска , бумага и холст повстречать там самый себя , идти восвояси , не получить ни единый цент по счёт ! '],
 6: ['И вот люди искусства на

In [124]:
to_upload_in_db = []

for k, v in sentences_in_text.items():
    for k1, v1 in v.items():
        to_upload_in_db.append((k, k1, v1[0], v1[1]))

In [131]:
to_upload_in_db[:5]

[(1,
  1,
  'В небольшом квартале к западу от Вашингтон-сквера улицы перепутались и переломались в короткие полоски, именуемые проездами.',
  'в небольшой квартал к запасть от вашингтон-сквер улица перепутаться и переломаться в короткий полоска , именовать проезд . '),
 (1,
  2,
  'Эти проезды образуют странные углы и кривые линии.',
  'этот проезд образовать странный угол и кривая линия . '),
 (1,
  3,
  'Одна улица там даже пересекает самое себя раза два.',
  'один улица там даже пересекать самый себя раз два . '),
 (1,
  4,
  'Некоему художнику удалось открыть весьма ценное свойство этой улицы.',
  'некий художник удаться открыть весьма ценный свойство этот улица . '),
 (1,
  5,
  'Предположим, сборщик из магазина со счетом за краски, бумагу и холст повстречает там самого себя, идущего восвояси, не получив ни единого цента по счету!',
  'предположим , сборщик из магазин с счёт за краска , бумага и холст повстречать там самый себя , идти восвояси , не получить ни единый цент по счёт 

In [176]:
import sqlite3
 
conn = sqlite3.connect("articles2.db")
cursor = conn.cursor()

#cursor.execute("""CREATE TABLE sentences
#                  (text_index integer, sentence_index integer, sentence text,
#                   lemmatized_sentence text)
#               """)


In [128]:
#cursor.executemany("INSERT INTO sentences VALUES (?,?,?,?)", to_upload_in_db)
#conn.commit()

## Добавляем в датабазу таблицу с заголовками

In [133]:
titles_indexes = []
for j in tqdm(range(len(poems_titles))): 
    titles_indexes.append((j+1, poems_titles[j]))

In [132]:
#cursor.execute("""CREATE TABLE titles
#                  (text_index integer, title text)
#               """)
#conn.commit()

In [136]:
#cursor.executemany("INSERT INTO titles VALUES (?,?)", titles_indexes)
#conn.commit()

## Создаем словарь вида {text_index: {sentence_index: [[word, pos, lemma], ...]}} и записываем в датабазу

In [161]:
def get_tags(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    poss = [word.pos for word in doc.tokens]
    words = [word.text for word in doc.tokens]
    lemmas = [morph.parse(word)[0].normal_form for word in words]
    return words, poss, lemmas

In [162]:
pos_tags_in_text = {}

for i in tqdm(range(len(poems))):    
    poem_tags = poems[i].split('  ') 
    poem_pos_tags = {}
    for j in range(len(poem_tags)):
        word_, pos_, lemma_ = get_tags(poem_tags[j])
        pos_tags = []
        for k in range(len(word_)):    
            pos_tags.append([word_[k].lower(), pos_[k], lemma_[k]])
        poem_pos_tags[j+1] = pos_tags
    pos_tags_in_text[i+1] = poem_pos_tags
    

In [163]:
pos_tags_in_text[1][1]

[['в', 'ADP', 'в'],
 ['небольшом', 'ADJ', 'небольшой'],
 ['квартале', 'NOUN', 'квартал'],
 ['к', 'ADP', 'к'],
 ['западу', 'NOUN', 'запасть'],
 ['от', 'ADP', 'от'],
 ['вашингтон-сквера', 'ADJ', 'вашингтон-сквер'],
 ['улицы', 'NOUN', 'улица'],
 ['перепутались', 'NOUN', 'перепутаться'],
 ['и', 'CCONJ', 'и'],
 ['переломались', 'NOUN', 'переломаться'],
 ['в', 'ADP', 'в'],
 ['короткие', 'ADJ', 'короткий'],
 ['полоски', 'NOUN', 'полоска'],
 [',', 'PUNCT', ','],
 ['именуемые', 'VERB', 'именовать'],
 ['проездами', 'NOUN', 'проезд'],
 ['.', 'PUNCT', '.']]

In [171]:
to_upload_in_db = []

for k, v in pos_tags_in_text.items():
    for k1, v1 in v.items():
        for i in range(len(v1)):
            to_upload_in_db.append((k, k1, i+1, v1[i][0], v1[i][2], v1[i][1]))

In [172]:
to_upload_in_db[:10]

[(1, 1, 1, 'в', 'в', 'ADP'),
 (1, 1, 2, 'небольшом', 'небольшой', 'ADJ'),
 (1, 1, 3, 'квартале', 'квартал', 'NOUN'),
 (1, 1, 4, 'к', 'к', 'ADP'),
 (1, 1, 5, 'западу', 'запасть', 'NOUN'),
 (1, 1, 6, 'от', 'от', 'ADP'),
 (1, 1, 7, 'вашингтон-сквера', 'вашингтон-сквер', 'ADJ'),
 (1, 1, 8, 'улицы', 'улица', 'NOUN'),
 (1, 1, 9, 'перепутались', 'перепутаться', 'NOUN'),
 (1, 1, 10, 'и', 'и', 'CCONJ')]

In [177]:
#cursor.execute("""CREATE TABLE pos_tags
#                  (text_index integer, sentence_index integer, word_index integer, word text,
#                   lemma text, pos text)
#               """)
#conn.commit()

In [178]:
#cursor.executemany("INSERT INTO pos_tags VALUES (?,?,?,?,?,?)", to_upload_in_db)
#conn.commit()